<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/seqtoseq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. INSTALL LIBRARIES AND PACKAGES

In [1]:

# Import the necessary libraries
!pip install datasets
!pip install transformers
!pip install torch
!pip install evaluate
!pip install jiwer
!pip install transformers[torch]
!pip install numpy
!pip install numpy==1.22.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 812.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 87.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not insta

In [2]:
##Imports required
import numpy as np
from datasets import load_dataset, Audio, DatasetDict
import torch
import evaluate
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import Seq2SeqTrainingArguments,Seq2SeqTrainer
#from transformers import TrainingArguments, Trainer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## 2. IMPORT DATA

In [3]:
dataset = DatasetDict()

# Load the PolyAI dataset.

dataset = load_dataset("PolyAI/minds14", name="en-US", split="train[:80]")

# Remove unnecessary columns


# Split the datasedataset =
dataset.remove_columns(['path','english_transcription','intent_class','lang_id'])

#split into train and test
dataset = dataset.train_test_split(test_size = 0.2, shuffle=False)


Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
#dataset['train']['audio'][0]

In [5]:
## Resample the dataset to 16 Khz as MCTCT model is trained on 16khz
dataset['train'] = dataset['train'].cast_column("audio", Audio(sampling_rate=16000))
dataset['test'] = dataset['test'].cast_column("audio", Audio(sampling_rate=16000))
# #dataset['train'] = dataset['train'].rename_column("audio", Audio(sampling_rate=16000))
# dataset['train'] = dataset['train'].rename_column("transcription","sentence")
# dataset['test'] = dataset['test'].rename_column("transcription","sentence")

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 64
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 16
    })
})

## 3. DATA PREPROCESSING

In [7]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en",task="transcribe", model_max_length=225)
model = WhisperForConditionalGeneration.from_pretrained('openai/whisper-tiny.en')
model.to(device)


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 384)
      (layers): ModuleList(
        (0-3): 4 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=384, out_features=384, bias=False)
            (v_proj): Linear(in_features=384, out_features=384, bias=True)
            (q_proj): Linear(in_features=384, out_features=384, bias=True)
            (out_proj): Linear(in_features=384, out_features=384, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (fin

In [8]:
# Preparing a function to process the entire dataset
# We need to crate two variables with name 'input_featrues'(input array of sound wave in raw foram) and 'labels'(transcription)

def prepare_dataset(batch):
    audio = batch["audio"]

   # batch["input_ids"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"],return_tensor = "pt").input_features[0]
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"],return_tensor = "pt").input_features[0]

    #processor.feature_extractor.pad(input_features, return_tensors="pt")
  #  with processor.as_target_processor():
    batch["labels"] = processor.tokenizer(batch["transcription"]).input_ids
    return batch

In [9]:
np.object = object
encoded_dataset = dataset.map(prepare_dataset,num_proc=4)

Map (num_proc=4):   0%|          | 0/64 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/16 [00:00<?, ? examples/s]

In [10]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id', 'input_features', 'labels'],
        num_rows: 64
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id', 'input_features', 'labels'],
        num_rows: 16
    })
})

In [11]:
# Creating a DataCollatorClass

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: processor
    padding: Union[bool, str] = "longest"

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
#     def __call__(self, features):# List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
       # print(features)
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels
       # print(batch)
        return batch #batch


In [12]:

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [13]:

# Evalution metric- We will be evaluating our model on word error rate

import evaluate

metric = evaluate.load("wer")


def compute_metrics(pred):
 #   wer = evaluate.load("wer")
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer =  metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


#### 4. Training the model

In [14]:

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
training_args = Seq2SeqTrainingArguments(
    output_dir="seqtoseq-trained",
    gradient_checkpointing=True,
    per_device_train_batch_size=2,
    learning_rate=1e-5,
   # warmup_steps=2,
    max_steps=500,
    fp16=True ,#False ,#True,
    optim='adafactor',
   # group_by_length=True,
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    eval_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    report_to = ["tensorboard"],
    #data_parallel=False
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    #data_parallel=False
  #  sampler = None
)


In [15]:
# Requires GPU for training
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
100,No log,0.838965,0.782258
200,No log,0.358441,7.766129
300,No log,0.197193,7.209677
400,No log,0.188182,12.758065
500,0.344500,0.188641,9.725806


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


TrainOutput(global_step=500, training_loss=0.34453616333007814, metrics={'train_runtime': 285.4278, 'train_samples_per_second': 3.504, 'train_steps_per_second': 1.752, 'total_flos': 2.461888512e+16, 'train_loss': 0.34453616333007814, 'epoch': 15.62})

In [16]:
## getting test data
inputs = processor(dataset['test'][8]["audio"]["array"], sampling_rate=16000, return_tensors="pt").to(device).input_features
print(f"The input test audio is: {dataset['test'][8]['transcription']}")

generated_ids = model.generate(inputs=inputs)
# prediction for test data
# with torch.no_grad():
#     logits = model(**i2.to(device)).logits

#predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(f'The output prediction is : {transcription}')


The input test audio is: how much do I have in my account
The output prediction is : 'm much do I have in my account
